<a href="https://colab.research.google.com/github/HereBeCode/StanceDetectionML-Brockport/blob/main/MultiClass_YT_Comment_Sentiment_Classification_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install / Import Modules

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 4.9 MB/s 
     |████████████████████████████████| 212 kB 70.1 MB/s 
     |████████████████████████████████| 101 kB 13.5 MB/s 
     |████████████████████████████████| 140 kB 66.4 MB/s 
     |████████████████████████████████| 1.1 MB 60.7 MB/s 
     |████████████████████████████████| 596 kB 61.5 MB/s 
     |████████████████████████████████| 127 kB 68.6 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
     |████████████████████████████████| 144 kB 72.4 MB/s 
     |████████████████████████████████| 271 kB 78.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dep

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.0 MB/s 
     |████████████████████████████████| 6.6 MB 56.7 MB/s 


In [ ]:
import re
from datasets import Dataset
import pandas as pd
from datasets import load_dataset

# Prepare Dataset

In [ ]:
# filename = "/content/DataTest.csv"
# df = pd.read_csv(filename, usecols=["commentTextDisplay", "label"], encoding="utf-8")
filename = "./DataAbortion.csv"
df = pd.read_csv(filename, usecols=['commentTextDisplay','label'], encoding='utf-8')
print(df)

df = df[(df.label == 0) | (df.label == 1) | (df.label == 2)]
df = df.astype({'label': int})
print(df)

                                      commentTextDisplay  label
0      A woman should have a choice. A child doesn’t ...    1.0
1      What’s extraordinary is that these men assume ...    1.0
2                                           baby killers    2.0
3      I&#39;m pro choice would I get a abortion myse...    1.0
4      Abortion is first degree murder, rightfully pu...    2.0
...                                                  ...    ...
81485                                            Finally    NaN
81486  What people don’t realize is that although the...    1.0
81487  &quot;Don&#39;t they deserve rights?&quot; But...    1.0
81488  Just let a woman HAVE the CHOICE to have an ab...    1.0
81489  Humans will never be on the same page on every...    1.0

[81490 rows x 2 columns]
                                      commentTextDisplay  label
0      A woman should have a choice. A child doesn’t ...      1
1      What’s extraordinary is that these men assume ...      1
2             

In [ ]:
def remove_html_and_other(text):
    new_text = re.sub(r'<a href.*\/a>', ' ', text)
    new_text = (new_text.replace('<br /', ' ').
                replace('<b>', ' ').
                replace('</b>', ' ').
                replace('&#39;', "\u0027").
                replace('<br >', ' ').
                replace('<br>', ' ').
                replace('\u2026', ' ').
                replace('&quot;', '\u0022').
                replace('1st', 'first ').
                replace('2nd', 'second ').
                replace('3rd', 'third ').
                replace('100%', 'one hundred percent ')
    )
    return new_text

def cleanTxt(text):
    TEXT = text.lower()
    TEXT = remove_html_and_other(TEXT)
    return TEXT

In [ ]:
df['commentTextDisplay'] = df['commentTextDisplay'].apply(cleanTxt)
print(df)

                                      commentTextDisplay  label
0      a woman should have a choice. a child doesn’t ...      1
1      what’s extraordinary is that these men assume ...      1
2                                           baby killers      2
3      i'm pro choice would i get a abortion myself p...      1
4      abortion is first degree murder, rightfully pu...      2
...                                                  ...    ...
81417  i'm still not sure if i'm pro-life or pro-choi...      0
81486  what people don’t realize is that although the...      1
81487  "don't they deserve rights?" but ur taking awa...      1
81488  just let a woman have the choice to have an ab...      1
81489  humans will never be on the same page on every...      1

[4284 rows x 2 columns]


In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 1000)
print(train_df)
print(test_df)

                                     commentTextDisplay  label
3850  abortion should be legal because a woman shoul...      1
871   not your body not your choice  gods body  god’...      2
359   i am pro-life- "pro-choice". you have the choi...      2
7229                                support the unborn!      2
2034  such an incredibly sad reflection of where  st...      2
...                                                 ...    ...
769   i stepped out of my own head and really listen...      2
1372  ok sorry how many different forms of birth con...      2
2398     let’s just allow women to make that choice smh      1
3776  all of this is shit. not a lot is taken into c...      1
1459  technically all living humans are clumps of ce...      0

[3427 rows x 2 columns]
                                     commentTextDisplay  label
3243            abortion is murder. you’re ending life.      2
1365  sorry, not sorry but if you can't keep it in y...      2
742   people need to stop with

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
print(train_dataset)
print(test_dataset)

print(train_dataset.features)

Dataset({
    features: ['commentTextDisplay', 'label', '__index_level_0__'],
    num_rows: 3427
})
Dataset({
    features: ['commentTextDisplay', 'label', '__index_level_0__'],
    num_rows: 857
})
{'commentTextDisplay': 'abortion should be legal because a woman should have the right to choose for her body. even though i am pro-choice, i actually hate abortion. i wish it didn\'t ever have to happen, but that\'s in a perfect world. instead of wasting time trying to forcibly "stop abortions" by changing laws, let\'s focus our efforts and money on prevention. we need better realistic sex education, better access to or free birth control, and raise a generation of people who will not rape (by this i mean poking holes in condoms and secretly taking off condoms as well). don\'t call yourself pro-life if you\'re not going to support real methods of prevention because at that point it\'s just about controlling women\'s sexuality.', 'label': 1, '__index_level_0__': 3850}
{'commentTextDisplay':

# Pre-Processing Dataset

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["commentTextDisplay"], truncation=True)

In [ ]:
tokenized_dataset = my_dataset.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7f111a57b170> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_train_dataset[0]

{'__index_level_0__': 3850,
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'commentTextDisplay': 'abortion should be legal because a woman should have the right to choose for her body. even though i am pro-choice, i actually hate abortion. i 

In [ ]:
tokenized_dataset['train'][10]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1],
 'commentTextDisplay': 'it’s murder.',
 'input_ids': [101, 2009, 1521, 1055, 4028, 1012, 102],
 'label': 2}

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
tf_train_set = tokenized_train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = tokenized_test_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator
)

In [ ]:
for element in tf_train_set:
  print(element)

{'input_ids': <tf.Tensor: shape=(16, 101), dtype=int64, numpy=
array([[  101,  2035,  3268, ...,     0,     0,     0],
       [  101,  2009,  1005, ...,     0,     0,     0],
       [  101,  1045,  2572, ...,     0,     0,     0],
       ...,
       [  101,  1045,  2228, ...,     0,     0,     0],
       [  101,  2367,  5711, ...,     0,     0,     0],
       [  101, 11324,  2003, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(16, 101), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'labels': <tf.Tensor: shape=(16,), dtype=int64, numpy=array([2, 0, 0, 1, 1, 1, 2, 1, 2, 2, 0, 1, 1, 0, 0, 1])>}
{'input_ids': <tf.Tensor: shape=(16, 189), dtype=int64, numpy=
array([[  101, 11324,  2003, ...,     0,     0,     0],
       [  101,  2074, 12731, ...,     0,     0,     0],
       [  101,  1996,  40

# Prepare Model

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_39', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer,
              metrics=['accuracy'])

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,779
Trainable params: 66,955,779
Non-trainable params: 0
_________________________________________________________________


# Fit/Fine-Tune Pre-Trained Model with Dataset

In [ ]:
history = model.fit(x=tf_train_set, 
          validation_data=tf_validation_set, 
          epochs=5)

Epoch 1/5
215/215 [==============================] - 105s 378ms/step - loss: 0.6764 - accuracy: 0.7146 - val_loss: 0.4959 - val_accuracy: 0.8156
Epoch 2/5
215/215 [==============================] - 82s 381ms/step - loss: 0.3471 - accuracy: 0.8783 - val_loss: 0.4056 - val_accuracy: 0.8425
Epoch 3/5
215/215 [==============================] - 79s 368ms/step - loss: 0.1642 - accuracy: 0.9519 - val_loss: 0.4583 - val_accuracy: 0.8565
Epoch 4/5
215/215 [==============================] - 80s 373ms/step - loss: 0.0864 - accuracy: 0.9767 - val_loss: 0.5330 - val_accuracy: 0.8378
Epoch 5/5
215/215 [==============================] - 79s 370ms/step - loss: 0.0545 - accuracy: 0.9860 - val_loss: 0.5830 - val_accuracy: 0.8355


# Model Analysis

In [ ]:
print(history)
history.validation_data

# Predict Sentiment with Single User Input

In [ ]:
# print("Enter your statement: ")
# input = [input()]

Enter your statement: 


TypeError: ignored

In [ ]:
input = ["abortion should never be allowed except in the case of rape."]

In [ ]:
input_df = pd.DataFrame(input, columns=['commentTextDisplay'])
input_dataset = Dataset.from_pandas(input_df)
tokenized_input = input_dataset.map(preprocess_function, batched=True)
input_ds = tokenized_input.to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    batch_size = 1
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
import numpy as np

prediction = model.predict(input_ds)
prediction_label = np.argmax(prediction.logits, axis=1)
print(prediction.logits)

[[ 3.5473285 -1.9738048 -1.9871725]]


In [ ]:
def convert_label_to_string(label):
  if (label == 0):
    return "Neutral"
  elif (label == 1):
    return "Positive"
  else:
    return "Negative"

print("Your input: " + input[0] + "\tPrediction: " + convert_label_to_string(prediction_label))

Your input: abortion should never be allowed except in the case of rape.	Prediction: Neutral


# Save Model (Zip)

In [ ]:
#model.save('BERT_acc_0_5_acc_81')

In [ ]:
#!zip -r BERT_acc_0_5_acc_81.zip BERT_acc_0_5_acc_81